# Learning Analytics Package: Build Dimension Tables

Builds the dimension tables used for the Learning Analytics package **v1.0** dashboard, in the context of using the Higher Ed. test data from Microsoft Education Insights roster, activity, and Microsoft Graph meeting attendance data.

The following tables are created in each of the steps outlined below (and above the cells for enrichment/curation):

1. dim_Student,
2. dim_Student_lookup,
3. dim_Instructor,
4. dim_Section,
5. dim_Course,
6. dim_School,
7. dim_Meeting,
8. dim_AssignmentStatus,
9. dim_Assignment
10. dim_SignalType, and
11. dim_Date.

This package-notebook also uses two methods (defined and outlined below):
 - **_publish_to_stage2**: uses the OEA_py function *upsert* to land the current dataframe in stage2/Enriched
 - **publish**: uses the method above to land the package dimension tables in stage2, then writes the same table to stage3/Published. Delta checkpoints are landed in the respective stage2/Enriched directory.

In [2]:
workspace = 'dev'
insights_version = '1.14'

StatementMeta(spark3p3sm, 97, 3, Finished, Available)

In [3]:
%run OEA_py

StatementMeta(, 97, -1, Finished, Available)

2023-05-22 02:58:54,806 - OEA - INFO - Now using workspace: dev
2023-05-22 02:58:54,807 - OEA - INFO - OEA initialized.


In [4]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace(workspace)

StatementMeta(spark3p3sm, 97, 5, Finished, Available)

2023-05-22 03:00:06,285 - OEA - INFO - Now using workspace: dev


## 0.) Define the Publish Function

### Notes
 - Can go about this in 2 ways (*thinking the second makes more sense*): 
   1. Write a new function (publish) that reads in a set of dfs, the aggregations, etc. writes checkpoints out to a new folder in stage 2, and then writes updated schema to stage3
   2. Perform aggregations and write finalized table to stage2, then publish to stage3 (keeps checkpoints in stage2 folder).


In [6]:
def _publish_to_stage2(df, destination, pk):
    oea.upsert(df, destination, pk)

def publish(df, stage2_destination, stage3_destination, primary_key='id'):
    _publish_to_stage2(df, stage2_destination, primary_key)

    spark.sql("set spark.sql.streaming.schemaInference=true")
    streaming_df = spark.readStream.format('delta').load(oea.to_url(stage2_destination))
    # for more info on append vs complete vs update modes for structured streaming: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#basic-concepts
    query = streaming_df.writeStream.format('delta').outputMode('append').trigger(once=True).option('checkpointLocation', oea.to_url(stage2_destination) + '/_checkpoints')
    query = query.start(oea.to_url(stage3_destination))
    query.awaitTermination()   # block until query is terminated, with stop() or with error; A StreamingQueryException will be thrown if an exception occurs.
    number_of_new_inbound_rows = query.lastProgress["numInputRows"]
    logger.info(f'Number of new inbound rows processed: {number_of_new_inbound_rows}')
    logger.debug(query.lastProgress)
    return number_of_new_inbound_rows

StatementMeta(spark3p3sm, 97, 7, Finished, Available)

## 1.) Build dim_Student Table

Data aggregations and curation on Insights roster data: AADUser and Person (as well as others).

This table has one row per student in the Insights roster data for the education system, with details around:
 - student IDs (previously, student internal ID or PersonId from Person table), 
 - student names (masked), 
 - student educational-grade levels, 
 - student UserPrincipalNames (pseudonymized UPNs from the AADUser table),
 - student date of birth (from PersonDemographic table, currently hashed here), and
 - student address (currently, birth city and state - which is hashed for pseudo data - since Insights data doesn't hold current address data).

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Student```.

In [ ]:
# read in tables needed
dfInsights_aaduserpersonmapping = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadUserPersonMapping')
dfInsights_aaduser = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadUser')
dfInsights_person = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Person')
dfInsights_personOrgRole = oea.load('stage2/Refined/M365/v' + insights_version + '/general/PersonOrganizationRole')
dfInsights_refDefinition = oea.load('stage2/Refined/M365/v' + insights_version + '/general/RefDefinition')
# lookup(s)
dfInsights_persondemographic_np = oea.load('stage2/Refined/M365/v' + insights_version + '/sensitive/PersonDemographic_lookup')

StatementMeta(spark3p2med, 58, 5, Finished, Available)

In [ ]:
# idea is eventually to actually write the OEA standardized schema to stage2/Enriched, then perform the aggregations and cleaning, and then write to stage3/Published.
# NOTE: Update as needed; remove unused or irrelelvant columns.

# start with Insights roster tables: Person and join with PersonOrganizationRole - to create the start of the Student table
dfInsights = dfInsights_personOrgRole.join(dfInsights_person, dfInsights_personOrgRole.PersonId_pseudonym == dfInsights_person.Id_pseudonym, how='inner')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId')
# join Insights table with the RefDefinition table - to get the Role of each person, and filter by only student SIS data
dfInsights = dfInsights.join(dfInsights_refDefinition, dfInsights.RefRoleId == dfInsights_refDefinition.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Code', 'PersonRole')
dfStudent = dfInsights.filter(dfInsights['PersonRole'] == 'Student')
dfStudent = dfStudent.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'RefGradeLevelId')
# join Student table with RefDefinition table, again - to get the grade of each student
dfStudent = dfStudent.join(dfInsights_refDefinition, dfInsights.RefGradeLevelId == dfInsights_refDefinition.Id, how='inner')
dfStudent = dfStudent.withColumnRenamed('Code', 'StudentGrade')
dfStudent = dfStudent.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade')
# join Student table with the AADUserPersonMapping table - to get the External Student ID
dfInsights_aaduserpersonmapping = dfInsights_aaduserpersonmapping.withColumnRenamed('PersonId_pseudonym', 'StudentId_internal_pseudonym')
dfStudent = dfStudent.join(dfInsights_aaduserpersonmapping, dfStudent.PersonId_pseudonym == dfInsights_aaduserpersonmapping.StudentId_internal_pseudonym, how='inner')
dfStudent = dfStudent.withColumnRenamed('ObjectId_pseudonym', 'StudentId_external_pseudonym')
dfStudent = dfStudent.select('StudentId_internal_pseudonym', 'StudentId_external_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade')
# join Student table with AAD User table to extract each student's UPN, and remove unneeded columns 
df_metadata = dfInsights_aaduser.select('ObjectId_pseudonym', 'UserPrincipalName_pseudonym')
dfStudent = dfStudent.join(df_metadata, dfStudent.StudentId_external_pseudonym == df_metadata.ObjectId_pseudonym, how='inner')
dfStudent = dfStudent.drop('ObjectId_pseudonym', 'PersonRole')
# join with Person Demographic table to extract each student's DoB and birth location (since current address isn't available from Insights roster data)
# also drop student external IDs (AAD IDs), and use Person ID as the primary key
df_metadata = dfInsights_persondemographic_np.select('PersonId_pseudonym', 'BirthDate', 'BirthCity', 'BirthState')
df_metadata = df_metadata.filter(df_metadata['BirthDate'] != 'BirthDate')
dfStudent = dfStudent.join(df_metadata, dfStudent.StudentId_internal_pseudonym == df_metadata.PersonId_pseudonym, how='inner')
dfStudent = dfStudent.drop('PersonId_pseudonym', 'StudentId_external_pseudonym').withColumnRenamed('StudentId_internal_pseudonym', 'StudentId_pseudonym')
display(dfStudent.limit(10))

StatementMeta(spark3p2med, 58, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 389813b4-c8d4-4382-bdea-a2690818eedc)

### Write to Stage 3

In [ ]:
publish(dfStudent, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Student', 'stage3/Published/learning_analytics/v1.0/general/dim_Student', primary_key='StudentId_pseudonym')

StatementMeta(spark3p2med, 58, 14, Finished, Available)

2023-01-11 18:10:06,073 - OEA - INFO - Number of new inbound rows processed: 600


600

## 2.) Build dim_Student_lookup Table

Data aggregations and curation on previously created dim_Student table and Insights roster data: Person_lookup (as well as others).

This table has one row per student in the Insights roster data, with details around:
 - student IDs (previously, student internal ID or PersonId from Person table; contains both hashed-pseudonym IDs and unhashed IDs), 
 - student UserPrincipalNames (contains both hashed-pseudonym UPNs and unhashed UPNs), and
 - student names (unmasked).

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/sensitive/dim_Student_lookup```.

In [ ]:
dfInsights_person_np = oea.load('stage2/Refined/M365/v'+ insights_version +'/sensitive/Person_lookup')
dfInsights_aaduser_np = oea.load('stage2/Refined/M365/v' + insights_version + '/sensitive/AadUser_lookup')

StatementMeta(spark3p2med, 58, 15, Finished, Available)

In [ ]:
# isolate relevant fields from dfStudent, and join with Person_lookup to get unhashed IDs as well as student names
dfStudent_lookup = dfStudent.select('StudentId_pseudonym', 'UserPrincipalName_pseudonym')
dfStudent_lookup = dfStudent_lookup.join(dfInsights_person_np, dfStudent_lookup.StudentId_pseudonym == dfInsights_person_np.Id_pseudonym, how='inner').drop('Id_pseudonym')
dfStudent_lookup = dfStudent_lookup.select('StudentId_pseudonym','Id','Surname','GivenName','MiddleName','UserPrincipalName_pseudonym').withColumnRenamed('Id', 'StudentId')
# join dfStudent with AADUser_lookup unhashed UPNs
dfInsights_aaduser_np_ = dfInsights_aaduser_np.select('UserPrincipalName', 'UserPrincipalName_pseudonym').withColumnRenamed('UserPrincipalName_pseudonym', 'UPN_pseudo')
dfStudent_lookup = dfStudent_lookup.join(dfInsights_aaduser_np_, dfStudent_lookup.UserPrincipalName_pseudonym == dfInsights_aaduser_np_.UPN_pseudo, how='inner').drop('UPN_pseudo')
display(dfStudent_lookup.limit(10))

StatementMeta(spark3p2med, 58, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9e25ee00-da5a-4a46-97de-0abfeab26a03)

### Write to Stage 3

In [ ]:
publish(dfStudent_lookup, 'stage2/Enriched/learning_analytics/v1.0/sensitive/dim_Student_lookup', 'stage3/Published/learning_analytics/v1.0/sensitive/dim_Student_lookup', primary_key='StudentId_pseudonym')

StatementMeta(spark3p2med, 58, 17, Finished, Available)

2023-01-11 18:12:57,409 - OEA - INFO - Number of new inbound rows processed: 600


600

## 3.) Build dim_Instructor Table

Data aggregations and curation on Insights activity and roster data: activity and Person (as well as others). *Note*: This only grabs the instructors listed in the Insights activity data; update for production purposes.

This table has one row per instructor in the Insights roster data for the education system, with details around:
 - instructor IDs (previously, instructor internal ID or Person ID from the Person table), and
 - instructor names.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Instructor```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')
dfInsights_aaduser = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadUser')
dfInsights_aaduserpersonmapping = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadUserPersonMapping')
# lookup(s)
dfInsights_aaduser_np = oea.load('stage2/Refined/M365/v' + insights_version + '/sensitive/AadUser_lookup')

StatementMeta(spark3p2med, 58, 18, Finished, Available)

In [ ]:
# NOTE: Clean up this to only do the processes needed
"""CONSIDER CREATING INSTRUCTOR GENERAL vs. LOOKUP"""
# extract the classes/instructors initally provided from the Insights Activity test data, and extract the Instructor IDs, number of sections that the instructor currently teaches.
dfInstructor = dfInsights_activity.filter(dfInsights_activity['ActorRole'] != 'Student')
dfInstructor = dfInstructor.select('ClassId', 'ActorId_pseudonym', 'ActorRole')
dfInstructor = dfInstructor.distinct()
dfInstructor = dfInstructor.groupBy('ActorId_pseudonym').count()
dfInstructor = dfInstructor.withColumnRenamed('ActorId_pseudonym', 'InstructorId_external_pseudonym').withColumnRenamed('count', 'NumSectionsCurrentlyInstructed')
# use the AADUser_pseudo table to get the AAD User Object ID of each professor (to get their name)
df_metadata = dfInsights_aaduser.filter(dfInsights_aaduser['Role'] != 'student')
df_metadata = df_metadata.select('ObjectId_pseudonym', 'TeacherId_pseudonym', 'Role')
dfInstructor = dfInstructor.join(df_metadata, dfInstructor.InstructorId_external_pseudonym == df_metadata.TeacherId_pseudonym, how='left')
dfInstructor = dfInstructor.drop('TeacherId_pseudonym').withColumnRenamed('Role', 'InstructorRole')
# use the AADUser_lookup table to get each professor's name
df_metadata2 = dfInsights_aaduser_np.select('UserPrincipalName', 'ObjectId_pseudonym').withColumnRenamed('ObjectId_pseudonym', 'id')
dfInstructor = dfInstructor.join(df_metadata2, dfInstructor.ObjectId_pseudonym == df_metadata2.id, how='inner')
dfInstructor = dfInstructor.drop('id').withColumnRenamed('UserPrincipalName', 'InstructorName')
# add each instructor's hashed internal ID (i.e. ID from Insights Person table)
df_metadata = dfInsights_aaduserpersonmapping.select('ObjectId_pseudonym', 'PersonId_pseudonym').withColumnRenamed('ObjectId_pseudonym', 'id')
dfInstructor = dfInstructor.join(df_metadata, dfInstructor.ObjectId_pseudonym == df_metadata.id, how='inner')
dfInstructor = dfInstructor.withColumnRenamed('PersonId_pseudonym', 'InstructorId_internal_pseudonym')
dfInstructor = dfInstructor.select('InstructorId_external_pseudonym', 'InstructorId_internal_pseudonym', 'InstructorRole', 'InstructorName', 'NumSectionsCurrentlyInstructed')
display(dfInstructor.limit(10))

StatementMeta(spark3p2med, 58, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, b098aa1a-8ec1-4b5a-bc07-9b2afa12f79d)

In [ ]:
# ad hoc - remove unused columns
dfInstructor = dfInstructor.select('InstructorId_internal_pseudonym', 'InstructorName').withColumnRenamed('InstructorId_internal_pseudonym', 'InstructorId_pseudonym')
display(dfInstructor.limit(10))

StatementMeta(spark3p2med, 58, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, 50982658-a53f-46c3-a681-b25810a23887)

### Write to Stage 3

In [ ]:
publish(dfInstructor, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Instructor', 'stage3/Published/learning_analytics/v1.0/general/dim_Instructor', primary_key='InstructorId_pseudonym')

StatementMeta(spark3p2med, 58, 21, Finished, Available)

2023-01-11 18:16:07,628 - OEA - INFO - Number of new inbound rows processed: 40


40

## 4.) Build dim_Section Table

Data aggregations and curation on Insights roster data: Section and SectionSession (and others). 

This table has one row per section/class in the Insights Section table, with details around:
 - section ID, 
 - section name,
 - section start & end date, and
 - calendar cycle that this particular section belongs to.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Section```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')
dfInsights_aadgroup = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadGroup')
#dfInsights_aadgroupmembership = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadGroupMembership')
dfInsights_aaduserpersonmapping = oea.load('stage2/Refined/M365/v' + insights_version + '/general/AadUserPersonMapping')
dfInsights_enrollment = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Enrollment')
dfInsights_section = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Section')
dfInsights_sectionsession = oea.load('stage2/Refined/M365/v' + insights_version + '/general/SectionSession')
dfInsights_session = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Session')
# lookup(s)
dfInsights_aadgroup_np = oea.load('stage2/Refined/M365/v' + insights_version + '/sensitive/AadGroup_lookup')

StatementMeta(spark3p2med, 58, 22, Finished, Available)

In [ ]:
# NOTE: Clean up this to only do the processes needed
# extract the classes initally provided from the Insights Activity test data, and extract the Teacher IDs 
dfClass = dfInsights_activity.filter(dfInsights_activity['ActorRole'] != 'Student')
dfClass = dfClass.filter(dfClass['ActorRole'] != 'ActorRole')
dfClass = dfClass.select('ClassId', 'ActorId_pseudonym', 'ActorRole')
dfClass = dfClass.groupBy('ClassId', 'ActorId_pseudonym', 'ActorRole').count()
dfClass = dfClass.withColumnRenamed('ClassId', 'AADGroup_ClassId').withColumnRenamed('ActorId_pseudonym', 'InstructorId_external_pseudonym').withColumnRenamed('ActorRole', 'PersonRole')
dfClass = dfClass.drop('count')
# join the Insights AADGroup_pseudo and _lookup table to the dfClass table, to provide mapping of the hashed and non-hashed AADGroup Class IDs
dfInsights_aadgroup_np_ = dfInsights_aadgroup_np.select('ObjectId', 'DisplayName', 'ObjectId_pseudonym').withColumnRenamed('ObjectId', 'Id').withColumnRenamed('ObjectId_pseudonym', 'AADGroup_ClassId_pseudonym')
dfClass = dfClass.join(dfInsights_aadgroup_np_, dfClass.AADGroup_ClassId == dfInsights_aadgroup_np_.Id, how='inner')
dfClass = dfClass.drop('Id')
dfInsights_aadgroup_ = dfInsights_aadgroup.select('ObjectId_pseudonym', 'SectionId')
dfClass = dfClass.join(dfInsights_aadgroup_, dfClass.AADGroup_ClassId_pseudonym == dfInsights_aadgroup_.ObjectId_pseudonym, how='inner')
dfClass = dfClass.drop('ObjectId_pseudonym', 'PersonRole').withColumnRenamed('DisplayName', 'SectionName')
# then, use the Enrollment table to group by class/Section and count the number of students enrolled within each class
df_metadata = dfInsights_enrollment.select('PersonId_pseudonym', 'SectionId', 'RefSectionRoleId').withColumnRenamed('SectionId', 'id')
df_metadata = df_metadata.filter(df_metadata['RefSectionRoleId'] == 'Student')
df_metadata = df_metadata.groupBy('id').count()
df_metadata = df_metadata.withColumnRenamed('count', 'NumStudentsEnrolledInSection')
dfClass = dfClass.join(df_metadata, dfClass.SectionId == df_metadata.id, how='inner')
dfClass = dfClass.select('SectionId', 'SectionName', 'InstructorId_external_pseudonym', 'NumStudentsEnrolledInSection')
display(dfClass.limit(10))

StatementMeta(spark3p2med, 58, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 31bbb1f5-8bd1-4135-abc9-f81c87a083e6)

In [ ]:
# ad hoc - drop now-unused columns and join with SectionSession data
dfSection = dfClass.select('SectionId', 'SectionName')
df_metadata1 = dfInsights_sectionsession.select('SectionId', 'SessionId').withColumnRenamed('SectionId', 'id')
dfSection = dfSection.join(df_metadata1, dfSection.SectionId == df_metadata1.id, how='inner').drop('id')
df_metadata2 = dfInsights_session.select('Id', 'Name', 'StartDate', 'EndDate')
dfSection = dfSection.join(df_metadata2, dfSection.SessionId == df_metadata2.Id, how='inner').drop('SessionId', 'Id')
dfSection = dfSection.select('SectionId', 'SectionName', 'StartDate', 'EndDate', 'Name').withColumnRenamed('StartDate', 'SectionStartDate').withColumnRenamed('EndDate', 'SectionEndDate').withColumnRenamed('Name', 'CalendarCycle')
display(dfSection.limit(10))

StatementMeta(spark3p2med, 58, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, 982736f4-7e65-4989-b373-d870ab4fc78e)

### Write to Stage 3

In [ ]:
publish(dfSection, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Section', 'stage3/Published/learning_analytics/v1.0/general/dim_Section', primary_key='SectionId')

StatementMeta(spark3p2med, 58, 25, Finished, Available)

2023-01-11 18:18:39,226 - OEA - INFO - Number of new inbound rows processed: 87


87

## 5.) Build dim_Course Table

Data aggregations and curation on Insights roster data: Course and CourseGradeLevel (and others). 

This table has one row per course in the Insights Course table, with details around:
 - course ID, 
 - course name,
 - course grade level, and
 - the number of students enrolled in each course.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Course```.

In [ ]:
dfInsights_course = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Course')
dfInsights_coursegradelevel = oea.load('stage2/Refined/M365/v' + insights_version + '/general/CourseGradeLevel')
dfInsights_enrollment = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Enrollment')
dfInsights_refdefinition = oea.load('stage2/Refined/M365/v' + insights_version + '/general/RefDefinition')
dfInsights_section = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Section')

StatementMeta(spark3p2med, 58, 29, Finished, Available)

In [ ]:
# NOTE: Clean up this to only do the processes needed; CourseGradeLevel bug.
# grab each course's ID and name, and RefGradeLevelId
dfCourse = dfInsights_course.select('Id', 'Name')
df_metadata = dfInsights_coursegradelevel.select('CourseId', 'RefGradeLevelId')
df_metadata = df_metadata.filter(df_metadata['CourseId'] != 'CourseId')
dfCourse = dfCourse.join(df_metadata, dfCourse.Id == df_metadata.CourseId, how='inner').drop('CourseId')
# join class table with RefDefinition table to get the grade level of each course
df_metadata = dfInsights_refdefinition.select('Id', 'Code').withColumnRenamed('Id', 'refId')
dfCourse = dfCourse.join(df_metadata, dfCourse.RefGradeLevelId == df_metadata.refId, how='inner').drop('refId', 'RefGradeLevelId')
dfCourse = dfCourse.withColumnRenamed('Id', 'CourseId').withColumnRenamed('Name', 'CourseName').withColumnRenamed('Code', 'CourseGradeLevel')
display(dfCourse.limit(10))

StatementMeta(spark3p2med, 58, 27, Finished, Available)

SynapseWidget(Synapse.DataFrame, f02f70c3-f58f-4b11-b22c-e7fb58d51137)

In [ ]:
# NOTE: This will need to be updated for production purposes.
# add a column for the number of enrolled students - found through using the Insights Enrollment and Section tables
df_metadata = dfInsights_enrollment.select('PersonId_pseudonym', 'SectionId', 'RefSectionRoleId')
df_metadata = df_metadata.filter(df_metadata['RefSectionRoleId'] == 'Student')
df_metadata2 = dfInsights_section.select('Id', 'CourseId')
df_metadata = df_metadata.join(df_metadata2, df_metadata.SectionId == df_metadata2.Id, how='inner').drop('SectionId', 'Id')
df_metadata = df_metadata.withColumnRenamed('CourseId', 'id')
df_metadata = df_metadata.groupBy('id').count()
df_metadata = df_metadata.withColumnRenamed('count', 'EnrolledStudents')
dfCourse = dfCourse.join(df_metadata, dfCourse.CourseId == df_metadata.id, how='inner').drop('id')
print('Number of Courses (should be 87):')
print(dfCourse.count())
display(dfCourse.limit(10))

StatementMeta(spark3p2med, 58, 30, Finished, Available)

Number of Courses (should be 87):
87


SynapseWidget(Synapse.DataFrame, 9dba11c3-5f29-4c8c-b699-c972ad858b07)

### Write to Stage 3

In [ ]:
publish(dfCourse, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Course', 'stage3/Published/learning_analytics/v1.0/general/dim_Course', primary_key='CourseId')

StatementMeta(spark3p2med, 58, 31, Finished, Available)

2023-01-11 18:33:19,158 - OEA - INFO - Number of new inbound rows processed: 87


87

## 6.) Build dim_School Table

Data aggregations and curation on Insights roster data: Organization. 

This table has one row per school in the Insights Organization table, with details around:
 - school ID, 
 - school name,
 - country of location, and
 - latitude & longitude coordinates of school location.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_School```.

In [ ]:
dfInsights_org = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Organization')

StatementMeta(spark3p2med, 58, 32, Finished, Available)

In [ ]:
# isolate the relevant columns from the Organization table
dfSchool = dfInsights_org.select('Id', 'Name')
dfSchool = dfSchool.withColumnRenamed('Id', 'SchoolId').withColumnRenamed('Name', 'SchoolName')
# fill in US for country of location, temporarily - and add additional columns as placeholders for now
dfSchool = dfSchool.withColumn('Country', F.lit('United States of America')).withColumn('Latitude', F.lit('')).withColumn('Longitude', F.lit(''))
display(dfSchool)

StatementMeta(spark3p2med, 58, 33, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2c8a6f08-c3b4-41af-babb-470dc7e342c5)

### Write to Stage 3

In [ ]:
publish(dfSchool, 'stage2/Enriched/learning_analytics/v1.0/general/dim_School', 'stage3/Published/learning_analytics/v1.0/general/dim_School', primary_key='SchoolId')

StatementMeta(spark3p2med, 58, 34, Finished, Available)

2023-01-11 18:34:10,951 - OEA - INFO - Number of new inbound rows processed: 5


5

## 7.) Build dim_Meeting Table

Data aggregations and curation on Insights activity data and Graph meeting_attendance_report pertaining to meetings. 

This table has one row per meeting recorded in both Insights and from the Graph query, with details around:
 - meeting ID,
 - date of the meeting,
 - meeting start time,
 - meeting end time, and
 - type of meeting.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Meeting```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')
dfGraph_meetingAtten = oea.load('stage2/Refined/graph_api/v1.0/general/meeting_attendance_report')

StatementMeta(spark3p2med, 58, 35, Finished, Available)

In [ ]:
# grab only the relevant data for the Meeting dimension table from Graph
dfMeeting = dfGraph_meetingAtten.select('meetingId', 'meetingStartDateTime', 'meetingEndDateTime')
dfMeeting = dfMeeting.groupBy('meetingId', 'meetingStartDateTime', 'meetingEndDateTime').count()
dfMeeting = dfMeeting.drop('count')
# grab only the relevant data for the Meeting table from Insights activity, and join the meeting type to the dfMeeting table
df_metadata = dfInsights_activity.select('MeetingSessionId', 'MeetingType')
df_metadata = df_metadata.groupBy('MeetingSessionId', 'MeetingType').count()
df_metadata = df_metadata.drop('count')
dfMeeting = dfMeeting.join(df_metadata, dfMeeting.meetingId == df_metadata.MeetingSessionId, how='inner').drop('MeetingSessionId')
dfMeeting = dfMeeting.withColumnRenamed('meetingId', 'MeetingId').withColumnRenamed('meetingStartDateTime', 'StartTime').withColumnRenamed('meetingEndDateTime', 'EndTime')
# add an additional column for the date of the meeting, and make the time columns only contain the times, without dates
dfMeeting = dfMeeting.withColumn('MeetingDate', F.to_date(F.col('StartTime')))
dfMeeting = dfMeeting.withColumn('STime', F.date_format('StartTime', 'HH:mm:ss')).withColumn('ETime', F.date_format('EndTime', 'HH:mm:ss')).drop('StartTime', 'EndTime')
dfMeeting = dfMeeting.withColumnRenamed('STime', 'StartTime').withColumnRenamed('ETime', 'EndTime')
dfMeeting = dfMeeting.select('MeetingId', 'MeetingDate', 'StartTime', 'EndTime', 'MeetingType')
display(dfMeeting.limit(10))

StatementMeta(spark3p2med, 58, 36, Finished, Available)

SynapseWidget(Synapse.DataFrame, aaf219c2-b679-4cb4-add9-d8cebe2f8722)

### Write to Stage 3

In [ ]:
publish(dfMeeting, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Meeting', 'stage3/Published/learning_analytics/v1.0/general/dim_Meeting', primary_key='MeetingId')

StatementMeta(spark3p2med, 58, 37, Finished, Available)

2023-01-11 18:39:35,853 - OEA - INFO - Number of new inbound rows processed: 261


261

## 8.) Build dim_AssignmentStatus Table

Data aggregations and curation on Insights activity data pertaining to the 4 possible assignment status's seen from the Insights activity table.

This table has one row per assignment status from the activity table, with details on what the encoded AssignmentStatusId represents.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_AssignmentStatus```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')

StatementMeta(spark3p2med, 58, 38, Finished, Available)

In [ ]:
# build assignment status table, based on the possible assignment actions from the Insights activity table
dfAssignStatus = dfInsights_activity.select('AppName', 'Action')
dfAssignStatus = dfAssignStatus.filter(dfAssignStatus['AppName'] == 'Assignments')
dfAssignStatus = dfAssignStatus.groupBy('Action').count()
dfAssignStatus = dfAssignStatus.withColumn('AssignmentStatusId', F.when(F.col('Action') == 'Assigned', '1').otherwise(F.when(F.col('Action') == 'Visited', '2').otherwise(F.when(F.col('Action') == 'Submitted', '3').otherwise(F.when(F.col('Action') == 'Returned', '4')))))
dfAssignStatus = dfAssignStatus.drop('count').withColumnRenamed('Action', 'AssignmentStatus')
display(dfAssignStatus.limit(10))

StatementMeta(spark3p2med, 58, 39, Finished, Available)

SynapseWidget(Synapse.DataFrame, eae073d2-6fda-4d00-8418-9cc3c6acf63b)

### Write to Stage 3

In [ ]:
publish(dfAssignStatus, 'stage2/Enriched/learning_analytics/v1.0/general/dim_AssignmentStatus', 'stage3/Published/learning_analytics/v1.0/general/dim_AssignmentStatus', primary_key='AssignmentStatusId')

StatementMeta(spark3p2med, 58, 40, Finished, Available)

2023-01-11 18:42:02,894 - OEA - INFO - Number of new inbound rows processed: 4


4

## 9.) Build dim_Assignment Table
Data aggregations and curation on Insights activity data pertaining to assignments. 

This table has one row per assignments from the Insights activity data, with details around the assigned and due dates.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Assignment```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')

In [ ]:
dfAssign = dfInsights_activity.filter(dfInsights_activity['Action'] == 'Assigned')
dfAssign = dfAssign.groupBy('AssignmentId', 'DueDate', 'StartTime').count()
dfAssign = dfAssign.drop('count').withColumn('AssignedDate', F.to_date(F.col('StartTime'))).withColumn('DueDate', F.to_date(F.col('DueDate'))).drop('StartTime')
print('Number of Assignments Recorded:')
print(dfAssign.count())
display(dfAssign.limit(10))

In [ ]:
dfTest = dfInsights_activity.groupBy('AssignmentId').count()
print(dfTest.count())

### Write to Stage 3

In [ ]:
publish(dfAssign, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Assignment', 'stage3/Published/learning_analytics/v1.0/general/dim_Assignment', primary_key='AssignmentId')

## 10.) Build dim_SignalType Table

Data aggregations and curation on Insights activity data pertaining to the various Insights activity SignalTypes. 

This table has one row per Insights SignalType from within the Activity data, with details and categorization of the SignalType.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_SignalType```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')

StatementMeta(spark3p2med, 58, 41, Finished, Available)

In [ ]:
# NOTE: Clean encoding process.
# build signal type table, based on the possible signal types from the Insights activity table
dfSignalType = dfInsights_activity.select('SignalType', 'AppName')
dfSignalType = dfSignalType.groupBy('SignalType').count()
# create a new column for categorizing the signals
def SignalCat(SignalType):
    if SignalType == 'PostChannelMessage':
        res = 'Messaging'
    elif SignalType == 'ReplyChannelMessage':
        res = 'Messaging'
    elif SignalType == 'VisitTeamChannel':
        res = 'Messaging'
    elif SignalType == 'ExpandChannelMessage':
        res = 'Messaging'
    elif SignalType == 'ReactedWithEmoji':
        res = 'Messaging'
    elif SignalType == 'Like':
        res = 'Files'
    elif SignalType == 'Unlike':
        res = 'Files'
    elif SignalType == 'FileAccessed':
        res = 'Files'
    elif SignalType == 'FileModified':
        res = 'Files'
    elif SignalType == 'FileDownloaded':
        res = 'Files'
    elif SignalType == 'FileUploaded':
        res = 'Files'
    elif SignalType == 'ShareNotificationRequested':
        res = 'Files'
    elif SignalType == 'AddedToSharedWithMe':
        res = 'Files'
    elif SignalType == 'CommentCreated':
        res = 'Files'
    elif SignalType == 'CommentDeleted':
        res = 'Files'
    elif SignalType == 'UserAtMentioned':
        res = 'Files'
    elif SignalType == 'Reflect':
        res = 'Reflect'
    elif SignalType == 'OneNotePageChanged':
        res = 'Notebook'
    elif SignalType == 'SubmissionEvent':
        res = 'Assignments'
    elif SignalType == 'AssignmentEvent':
        res = 'Assignments'
    elif SignalType == 'CallRecordSummarized':
        res = 'TeamsMeeting'
    else:
        res = ''
    return res

def SignalTypeId(SignalType):
    if SignalType == 'PostChannelMessage':
        res = '000001'
    elif SignalType == 'ReplyChannelMessage':
        res = '000002'
    elif SignalType == 'VisitTeamChannel':
        res = '000003'
    elif SignalType == 'ExpandChannelMessage':
        res = '000004'
    elif SignalType == 'ReactedWithEmoji':
        res = '000005'
    elif SignalType == 'Like':
        res = '000010'
    elif SignalType == 'Unlike':
        res = '000011'
    elif SignalType == 'FileAccessed':
        res = '000020'
    elif SignalType == 'FileModified':
        res = '000021'
    elif SignalType == 'FileDownloaded':
        res = '000022'
    elif SignalType == 'FileUploaded':
        res = '000023'
    elif SignalType == 'ShareNotificationRequested':
        res = '000030'
    elif SignalType == 'AddedToSharedWithMe':
        res = '000040'
    elif SignalType == 'CommentCreated':
        res = '000050'
    elif SignalType == 'CommentDeleted':
        res = '000051'
    elif SignalType == 'UserAtMentioned':
        res = '000060'
    elif SignalType == 'Reflect':
        res = '000100'
    elif SignalType == 'OneNotePageChanged':
        res = '001000'
    elif SignalType == 'SubmissionEvent':
        res = '020000'
    elif SignalType == 'AssignmentEvent':
        res = '010000'
    elif SignalType == 'CallRecordSummarized':
        res = '100000'
    else:
        res = ''
    return res
# define the function/dataType
new_f = F.udf(SignalCat, StringType())
new_f2 = F.udf(SignalTypeId, StringType())
  
# add the new column
dfSignalType = dfSignalType.withColumn("SignalCategory", new_f('SignalType'))
dfSignalType = dfSignalType.withColumn('SignalTypeId', new_f2('SignalType'))
dfSignalType = dfSignalType.drop('count')
display(dfSignalType)

StatementMeta(spark3p2med, 58, 42, Finished, Available)

SynapseWidget(Synapse.DataFrame, 27a7502d-cd08-4acc-bb9c-1d34fd86912a)

### Write to Stage 3

In [ ]:
publish(dfSignalType, 'stage2/Enriched/learning_analytics/v1.0/general/dim_SignalType', 'stage3/Published/learning_analytics/v1.0/general/dim_SignalType', primary_key='SignalTypeId')

StatementMeta(spark3p2med, 58, 43, Finished, Available)

2023-01-11 18:43:24,564 - OEA - INFO - Number of new inbound rows processed: 21


21

## 11.) Build dim_Date Table

Data aggregations and curation on Insights activity data pertaining to the various dates seen from activity. This table is used in the Power BI data model to connect dates seen tables, for filters in the dashboard.

This table has one row per date seen from the activity data used, with details on that particular date.

This table is then written out to```(stage2 and stage3)/(Enriched and Published)/learning_analytics/v1.0/general/dim_Date```.

In [ ]:
dfInsights_activity = oea.load('stage2/Refined/M365/v' + insights_version + '/general/activity')
dfInsights_session = oea.load('stage2/Refined/M365/v' + insights_version + '/general/Session')

StatementMeta(spark3p2med, 58, 44, Finished, Available)

In [ ]:
# NOTE: Double-check that all columns are relevant and used in the dashboard.
# extract dates to create Date table from Insights activity data
dfDate = dfInsights_activity.select('StartTime')
dfDate = dfDate.withColumn('Date', F.to_date(F.col('StartTime')))
dfDate = dfDate.groupBy('Date').count()
dfDate = dfDate.orderBy('Date')
dfDate = dfDate.drop('count').withColumn('Year', F.year(F.col('Date'))).withColumn('Month', F.month(F.col('Date')))
# extract the start date of the school year from the Insights Session table; in this case filter the table to get the start date of the school year
list_of_schoolStartDate = dfInsights_session.filter(dfInsights_session['Name'] == '2021-2022').select('StartDate').collect()
dfDate = dfDate.withColumn('SchoolYearStartDate', F.lit(list_of_schoolStartDate[0][0]))
dfDate = dfDate.withColumn('SchoolYearStartDate', F.to_date(F.col('SchoolYearStartDate')))
# calculate the number of months between activity date and the start date of the school year - then add 1 month of the floor to get the month of the school year
dfDate = dfDate.withColumn('MonthOfSchoolYear', F.months_between(F.col('Date'), F.col('SchoolYearStartDate')))
dfDate = dfDate.withColumn('MonthOfSchoolYear', F.floor(F.col('MonthOfSchoolYear'))).withColumn('MonthOfSchoolYear', F.col('MonthOfSchoolYear') + 1)
# calculate the number of days between date and start of school year - then take the floor of the number of weeks (add 1 week) to get the week number of the school year
dfDate = dfDate.withColumn('WeekOfSchoolYear', F.datediff(F.col('Date'), F.col('SchoolYearStartDate'))/7)
dfDate = dfDate.withColumn('WeekOfSchoolYear', F.floor(F.col('WeekOfSchoolYear'))).withColumn('WeekOfSchoolYear', F.col('WeekOfSchoolYear') + 1)
display(dfDate.limit(10))

StatementMeta(spark3p2med, 58, 45, Finished, Available)

SynapseWidget(Synapse.DataFrame, 04f636ed-316a-4a5e-a5d7-c9b4a4433db7)

In [ ]:
# find the start date of the semester
list_of_semesterStartDate = dfInsights_session.filter(dfInsights_session['Name'] == 'Spring2022').select('StartDate').collect()
dfDate = dfDate.withColumn('SemesterStartDate', F.lit(list_of_semesterStartDate[0][0]))
# calculate the number of months between activity date and the start date of the semester - then add 1 month of the floor to get the month of the semester
dfDate = dfDate.withColumn('MonthOfSemester', F.months_between(F.col('Date'), F.col('SemesterStartDate')))
dfDate = dfDate.withColumn('MonthOfSemester', F.floor(F.col('MonthOfSemester'))).withColumn('MonthOfSemester', F.col('MonthOfSemester') + 1)
# calculate the number of days between date and start of semester - then take the floor of the number of weeks (add 1 week) to get the week number of the semester
dfDate = dfDate.withColumn('WeekOfSemester', F.datediff(F.col('Date'), F.col('SemesterStartDate'))/7)
dfDate = dfDate.withColumn('WeekOfSemester', F.floor(F.col('WeekOfSemester'))).withColumn('WeekOfSemester', F.col('WeekOfSemester') + 1)
display(dfDate.limit(10))

StatementMeta(spark3p2med, 58, 46, Finished, Available)

SynapseWidget(Synapse.DataFrame, da904f7c-2d2f-462e-9a1b-8b4c72817a41)

### Write to Stage 3

In [ ]:
publish(dfDate, 'stage2/Enriched/learning_analytics/v1.0/general/dim_Date', 'stage3/Published/learning_analytics/v1.0/general/dim_Date', primary_key='Date')

StatementMeta(spark3p2med, 58, 47, Finished, Available)

2023-01-11 18:45:37,705 - OEA - INFO - Number of new inbound rows processed: 34


34